# Full Grid Prediction - Teams + Drivers

Complete prediction pipeline combining team and driver performance.

**Two-stage approach:**
1. **Team Performance** (car + track characteristics) → Team rankings 1-10 (or 1-11 for 2026)
2. **Driver Positions** (driver skill + team rank) → Full grid positions 1-20 (or 1-22 for 2026)

**Why both?**
- VER beats LAW (same car, better driver)
- NOR beats PIA (same car, slightly better pace)
- Team prediction shows car-track fit
- Driver prediction shows actual grid order

**Lineups:** 
- 2024-2025: Extracted from actual session data (handles all corner cases!)
- 2026+: Uses current_lineups.json (update when drivers change)

**2026 note:** First season with 11 teams (22 drivers) since Cadillac joins!

## Setup

In [12]:
import sys
import json
from pathlib import Path
import pandas as pd
import numpy as np

sys.path.append('../')

from src.predictors.team_predictor import rank_teams_for_track
from src.predictors.driver_predictor import DriverRanker
from src.utils.lineup_manager import get_lineups  # NEW!

import warnings
warnings.filterwarnings('ignore')

## Configuration

In [13]:
# Which season to predict/validate?
SEASON = 2025  # Change to 2026 for future predictions

# Grid size by season
GRID_SIZES = {
    2024: 20,
    2025: 20,
    2026: 22  # Cadillac joins!
}

EXPECTED_TEAMS = {
    2024: 10,
    2025: 10,
    2026: 11  # + Cadillac
}

print(f"Season: {SEASON}")
print(f"Expected teams: {EXPECTED_TEAMS[SEASON]}")
print(f"Expected drivers: {GRID_SIZES[SEASON]}")

Season: 2025
Expected teams: 10
Expected drivers: 20


## Load Data

In [14]:
# Track what loaded
loaded = []
errors = []

# Load track characteristics
try:
    track_path = Path(f'../data/processed/testing_files/track_characteristics/{SEASON}_track_characteristics.json')
    with open(track_path) as f:
        track_data = json.load(f)
    all_tracks = track_data.get('tracks', {})
    loaded.append(f"tracks ({len(all_tracks)})")
except FileNotFoundError:
    errors.append("track characteristics")
    all_tracks = {}

# Load car characteristics
try:
    car_path = Path(f'../data/processed/testing_files/car_characteristics/{SEASON}_car_characteristics.json')
    with open(car_path) as f:
        car_data = json.load(f)
    all_cars = car_data.get('teams', {})
    loaded.append(f"teams ({len(all_cars)})")
except FileNotFoundError:
    errors.append("car characteristics")
    all_cars = {}

# Load driver ranker
try:
    driver_ranker = DriverRanker(
        '../data/processed/testing_files/driver_characteristics/driver_characteristics.json'
    )
    loaded.append("driver ranker")
except FileNotFoundError:
    errors.append("driver characteristics")
    driver_ranker = None

# Load driver characteristics for reference
try:
    with open('../data/processed/testing_files/driver_characteristics/driver_characteristics.json') as f:
        driver_data = json.load(f)
    loaded.append(f"driver profiles ({driver_data.get('total_drivers', 0)})")
except FileNotFoundError:
    driver_data = {}

# Print summary
if loaded:
    print(f"🟢 Loaded: {', '.join(loaded)}")
if errors:
    print(f"🔴 Missing: {', '.join(errors)}")

Loaded characteristics for 27 drivers
🟢 Loaded: tracks (24), teams (10), driver ranker, driver profiles (27)


## Demo: Single Race Prediction

Show complete pipeline for one race.

**Lineups:**
- For 2024-2025: Extracted from actual session data (automatically handles reserves, swaps, etc.)
- For 2026+: Uses current_lineups.json

In [15]:
# Pick a race
demo_race = 'Bahrain Grand Prix'

if demo_race in all_tracks:
    track = all_tracks[demo_race]
    
    print(f"{'='*70}")
    print(f"FULL PREDICTION: {demo_race}")
    print(f"{'='*70}")
    
    # Get lineups (automatic!)
    current_lineups = get_lineups(SEASON, demo_race)
    
    print(f"\nTeam Lineups:")
    print("-" * 70)
    for team, drivers in sorted(current_lineups.items()):
        drivers_str = ' / '.join(drivers) if len(drivers) >= 2 else drivers[0]
        print(f"  {team:<25} {drivers_str}")
    
    # Stage 1: Predict team performance
    print(f"\nSTAGE 1: Team Performance (Post-FP3)")
    print("-" * 70)
    
    team_rankings = rank_teams_for_track(all_cars, track, 'post_fp3', 'normal')
    
    # Convert to dict for driver_ranker
    team_predictions = {team: rank for rank, (team, _, _, _) in enumerate(team_rankings, 1)}
    
    print(f"\n{'Pos':<5} {'Team':<25} {'Score':<10} {'Conf':<8}")
    for rank, (team, score, conf, _) in enumerate(team_rankings, 1):
        print(f"{rank:<5} {team:<25} {score:<10.3f} {conf:<8.2f}")
    
    # Stage 2: Convert to driver positions
    print(f"\nSTAGE 2: Driver Positions (Full Grid)")
    print("-" * 70)
    
    driver_results = driver_ranker.predict_positions(
        team_predictions=team_predictions,
        team_lineups=current_lineups,
        session_type='qualifying'
    )
    
    print(f"\n{driver_ranker.format_predictions(driver_results, top_n=GRID_SIZES[SEASON])}")
    
    # Show key insights
    print(f"\nKEY INSIGHTS:")
    print("-" * 70)
    
    # Teammate battles
    print(f"\nIntra-team battles:")
    for team, drivers in current_lineups.items():
        if len(drivers) < 2:
            continue
            
        d1_pred = next((p for p in driver_results['predictions'] if p.driver == drivers[0]), None)
        d2_pred = next((p for p in driver_results['predictions'] if p.driver == drivers[1]), None)
        
        if d1_pred and d2_pred:
            gap = abs(d1_pred.position - d2_pred.position)
            winner = drivers[0] if d1_pred.position < d2_pred.position else drivers[1]
            print(f"  {team:<25} {winner} ahead by {gap:.1f} positions")

else:
    print(f"Race '{demo_race}' not found in {SEASON} data")

FULL PREDICTION: Bahrain Grand Prix

Team Lineups:
----------------------------------------------------------------------
  Alpine                    GAS / DOO
  Aston Martin              ALO / STR
  Ferrari                   LEC / HAM
  Haas F1 Team              OCO / BEA
  Kick Sauber               HUL / BOR
  McLaren                   PIA / NOR
  Mercedes                  RUS / ANT
  Racing Bulls              HAD / LAW
  Red Bull Racing           VER / TSU
  Williams                  SAI / ALB

STAGE 1: Team Performance (Post-FP3)
----------------------------------------------------------------------

Pos   Team                      Score      Conf    
1     Mercedes                  0.399      0.85    
2     Red Bull Racing           0.383      0.85    
3     McLaren                   0.341      0.85    
4     Ferrari                   0.317      0.85    
5     Racing Bulls              0.292      0.85    
6     Williams                  0.290      0.85    
7     Aston Martin      

## Validation: Compare vs Actual Results

Only works for 2024-2025 where we have actual results.

In [16]:
if SEASON <= 2025:
    # Load actual results
    results_path = Path(f'../data/processed/testing_files/validation/{SEASON}_qualifying_results.json')
    
    if results_path.exists():
        with open(results_path) as f:
            actual_results = json.load(f)
        
        print(f"🟢 Loaded actual results for {actual_results['total_races']} races")
        print("\nValidation ready!")
    else:
        print(f"⚠️  No actual results found at {results_path}")
        print("Run notebook 10 to extract actual results first.")
        actual_results = None
else:
    print(f"Season {SEASON} is in the future - no actual results available")
    print("Predictions only mode.")
    actual_results = None

🟢 Loaded actual results for 24 races

Validation ready!


In [17]:
if actual_results and demo_race in actual_results['races']:
    actual = actual_results['races'][demo_race]
    
    print(f"\n{'='*70}")
    print(f"VALIDATION: {demo_race}")
    print(f"{'='*70}")
    
    # Compare predictions vs actuals
    print(f"\n{'Pos':<5} {'Predicted':<25} {'Actual':<25} {'Error':<8}")
    print("-" * 70)
    
    for i in range(min(10, GRID_SIZES[SEASON])):
        pred = driver_results['predictions'][i]
        actual_driver = actual['positions'][i]
        
        # Find actual position of our predicted driver
        actual_pos_of_pred = next(
            (p['position'] for p in actual['positions'] if p['driver'] == pred.driver),
            None
        )
        
        if actual_pos_of_pred:
            error = pred.position - actual_pos_of_pred
            error_str = f"{error:+.1f}"
        else:
            error_str = "N/A"
        
        print(f"{i+1:<5} {pred.driver} ({pred.team[:20]}){'':<3} "
              f"{actual_driver['driver']} ({actual_driver['team'][:20]}){'':<3} {error_str:<8}")
    
    # Calculate metrics
    errors = []
    for pred in driver_results['predictions']:
        actual_pos = next(
            (p['position'] for p in actual['positions'] if p['driver'] == pred.driver),
            None
        )
        if actual_pos:
            errors.append(abs(pred.position - actual_pos))
    
    if errors:
        mae = np.mean(errors)
        within_1 = sum(1 for e in errors if e <= 1) / len(errors)
        within_2 = sum(1 for e in errors if e <= 2) / len(errors)
        within_3 = sum(1 for e in errors if e <= 3) / len(errors)
        
        print(f"\n📊 ACCURACY:")
        print("-" * 70)
        print(f"  MAE:          {mae:.2f} positions")
        print(f"  ±1 position:  {within_1*100:.1f}%")
        print(f"  ±2 positions: {within_2*100:.1f}%")
        print(f"  ±3 positions: {within_3*100:.1f}%")

elif actual_results:
    print(f"\n🔴  No actual results for {demo_race}")
else:
    print(f"\n🔴  No actual results loaded - predictions only")


VALIDATION: Bahrain Grand Prix

Pos   Predicted                 Actual                    Error   
----------------------------------------------------------------------
1     RUS (Mercedes)    PIA (McLaren)    -1.0    
2     ANT (Mercedes)    RUS (Mercedes)    -2.0    
3     VER (Red Bull Racing)    LEC (Ferrari)    -4.0    
4     TSU (Red Bull Racing)    ANT (Mercedes)    -6.0    
5     PIA (McLaren)    GAS (Alpine)    +4.0    
6     NOR (McLaren)    NOR (McLaren)    +0.0    
7     LEC (Ferrari)    VER (Red Bull Racing)    +4.0    
8     HAM (Ferrari)    SAI (Williams)    -1.0    
9     HAD (Racing Bulls)    HAM (Ferrari)    -3.0    
10    LAW (Racing Bulls)    TSU (Red Bull Racing)    -7.0    

📊 ACCURACY:
----------------------------------------------------------------------
  MAE:          3.20 positions
  ±1 position:  35.0%
  ±2 positions: 40.0%
  ±3 positions: 55.0%


In [18]:
print("\nDIAGNOSTIC:")
print("="*70)

# Check team predictions
print("\nTeam rankings (predicted):")
for i, (team, score, _, _) in enumerate(team_rankings[:5], 1):
    print(f"  {i}. {team:<25} {score:.3f}")

# Check actual results by team
actual_teams = {}
for pos in actual['positions'][:10]:
    team = pos['team']
    if team not in actual_teams:
        actual_teams[team] = []
    actual_teams[team].append(pos['position'])

print("\nActual results (top teams):")
for team, positions in sorted(actual_teams.items(), key=lambda x: min(x[1]))[:5]:
    print(f"  {team:<25} Positions: {positions}")


DIAGNOSTIC:

Team rankings (predicted):
  1. Mercedes                  0.399
  2. Red Bull Racing           0.383
  3. McLaren                   0.341
  4. Ferrari                   0.317
  5. Racing Bulls              0.292

Actual results (top teams):
  McLaren                   Positions: [1, 6]
  Mercedes                  Positions: [2, 4]
  Ferrari                   Positions: [3, 9]
  Alpine                    Positions: [5]
  Red Bull Racing           Positions: [7, 10]


## Full Season Validation

Validate all races at both team and driver level.

**Uses dynamic lineups:** Each race gets the actual lineup from that session!

In [19]:
if actual_results:
    all_validations = []
    
    print(f"Running full season validation...")
    print(f"{'='*70}\n")
    
    for race_name, race_data in actual_results['races'].items():
        if race_name not in all_tracks:
            print(f"⚠️  {race_name}: No track data")
            continue
        
        track = all_tracks[race_name]
        weekend_type = race_data['weekend_type']
        
        # Get lineups for THIS specific race (handles mid-season changes!)
        lineups = get_lineups(SEASON, race_name)
        
        # Determine final session
        if weekend_type == 'sprint':
            final_session = 'post_sprint_quali'
        else:
            final_session = 'post_fp3'
        
        # Predict teams
        team_rankings = rank_teams_for_track(all_cars, track, final_session, weekend_type)
        if not team_rankings:
            continue
        
        team_predictions = {team: rank for rank, (team, _, _, _) in enumerate(team_rankings, 1)}
        
        # Predict drivers
        session_type = 'qualifying'
        driver_results = driver_ranker.predict_positions(
            team_predictions=team_predictions,
            team_lineups=lineups,  # Race-specific lineups!
            session_type=session_type
        )
        
        # Calculate errors
        errors = []
        for pred in driver_results['predictions']:
            actual_pos = next(
                (p['position'] for p in race_data['positions'] if p['driver'] == pred.driver),
                None
            )
            if actual_pos:
                errors.append(abs(pred.position - actual_pos))
        
        if errors:
            validation = {
                'race': race_name,
                'weekend_type': weekend_type,
                'mae': np.mean(errors),
                'within_1': sum(1 for e in errors if e <= 1) / len(errors),
                'within_2': sum(1 for e in errors if e <= 2) / len(errors),
                'within_3': sum(1 for e in errors if e <= 3) / len(errors),
            }
            all_validations.append(validation)
            print(f"🟢 {race_name:<30} MAE: {validation['mae']:.2f}")
    
    # Overall summary
    if all_validations:
        print(f"\n{'='*70}")
        print(f"OVERALL VALIDATION SUMMARY")
        print(f"{'='*70}")
        
        overall_mae = np.mean([v['mae'] for v in all_validations])
        overall_w1 = np.mean([v['within_1'] for v in all_validations])
        overall_w2 = np.mean([v['within_2'] for v in all_validations])
        overall_w3 = np.mean([v['within_3'] for v in all_validations])
        
        print(f"\nAcross {len(all_validations)} races:")
        print(f"  MAE:          {overall_mae:.2f} positions")
        print(f"  ±1 position:  {overall_w1*100:.1f}%")
        print(f"  ±2 positions: {overall_w2*100:.1f}%")
        print(f"  ±3 positions: {overall_w3*100:.1f}%")
        
        # By weekend type
        normal_vals = [v for v in all_validations if v['weekend_type'] == 'normal']
        sprint_vals = [v for v in all_validations if v['weekend_type'] == 'sprint']
        
        if normal_vals:
            print(f"\nNormal weekends ({len(normal_vals)} races):")
            print(f"  MAE: {np.mean([v['mae'] for v in normal_vals]):.2f}")
        
        if sprint_vals:
            print(f"\nSprint weekends ({len(sprint_vals)} races):")
            print(f"  MAE: {np.mean([v['mae'] for v in sprint_vals]):.2f}")

else:
    print("No actual results available for validation")

Running full season validation...

🟢 Australian Grand Prix          MAE: 3.30
🟢 Chinese Grand Prix             MAE: 3.10
🟢 Japanese Grand Prix            MAE: 3.60
🟢 Bahrain Grand Prix             MAE: 3.20
🟢 Saudi Arabian Grand Prix       MAE: 2.70
🟢 Miami Grand Prix               MAE: 3.70
🟢 Emilia Romagna Grand Prix      MAE: 4.20
🟢 Monaco Grand Prix              MAE: 4.40
🟢 Spanish Grand Prix             MAE: 3.90
🟢 Canadian Grand Prix            MAE: 3.20
🟢 Austrian Grand Prix            MAE: 4.40
🟢 British Grand Prix             MAE: 3.70
🟢 Belgian Grand Prix             MAE: 5.80
🟢 Hungarian Grand Prix           MAE: 5.20
🟢 Dutch Grand Prix               MAE: 3.20
🟢 Italian Grand Prix             MAE: 5.20
🟢 Azerbaijan Grand Prix          MAE: 3.00
🟢 Singapore Grand Prix           MAE: 3.10
🟢 United States Grand Prix       MAE: 4.20
🟢 Mexico City Grand Prix         MAE: 4.10
🟢 São Paulo Grand Prix           MAE: 5.30
🟢 Las Vegas Grand Prix           MAE: 5.30
🟢 Qatar Grand Prix 

## 2026 Predictions

When SEASON=2026, this shows predictions for the new regulation era.

**Update lineups:** Edit `data/current_lineups.json` when Cadillac announces drivers!

In [20]:
if SEASON == 2026:
    print(f"{'='*70}")
    print("2026 PREDICTIONS - NEW REGULATION ERA")
    print(f"{'='*70}")
    print(f"\n⚠️  2026 has new regulations:")
    print("  - New aerodynamics")
    print("  - New power units")
    print("  - Active aero systems")
    print("  - Cadillac joins (11 teams, 22 drivers)")
    print(f"\n💡 Predictions will have higher uncertainty due to regulation change!")
    
    # Show current lineups
    current_lineups_2026 = get_lineups(2026)
    print(f"\n📋 Current 2026 Lineups:")
    print("-" * 70)
    for team, drivers in sorted(current_lineups_2026.items()):
        drivers_str = ' / '.join(drivers)
        print(f"  {team:<25} {drivers_str}")
    
    print(f"\n📝 Update data/current_lineups.json when Cadillac announces drivers!")
    print(f"\nReady to make predictions once 2026 track/car data is available.")

## Export Predictions

Save predictions for use in other tools.

In [21]:
def save_race_prediction(race_name, team_predictions, driver_results, output_dir='../data/predictions'):
    """
    Save complete race prediction to JSON.
    """
    output = {
        'season': SEASON,
        'race': race_name,
        'team_predictions': team_predictions,
        'driver_predictions': [
            {
                'position': pred.position,
                'driver': pred.driver,
                'team': pred.team,
                'confidence_lower': pred.confidence_lower,
                'confidence_upper': pred.confidence_upper
            }
            for pred in driver_results['predictions']
        ]
    }
    
    output_path = Path(output_dir) / f"{SEASON}_{race_name.replace(' ', '_')}_prediction.json"
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"🟢 Saved prediction to {output_path}")
    return output_path

# Example usage
# save_race_prediction(demo_race, team_predictions, driver_results)

In [22]:
# After team predictions
print("\nTeam Prediction Check:")
print("-" * 70)

# Compare predicted teams vs actual top 3 drivers' teams
actual_top3_teams = [
    actual['positions'][0]['team'],
    actual['positions'][1]['team'],
    actual['positions'][2]['team']
]

predicted_top3_teams = [team for team, _, _, _ in team_rankings[:3]]

print(f"Predicted top 3 teams: {predicted_top3_teams}")
print(f"Actual top 3 teams:    {actual_top3_teams}")

# How many match?
matches = sum(1 for t in predicted_top3_teams if t in actual_top3_teams)
print(f"Matches: {matches}/3")


Team Prediction Check:
----------------------------------------------------------------------
Predicted top 3 teams: ['Mercedes', 'Red Bull Racing', 'McLaren']
Actual top 3 teams:    ['McLaren', 'Mercedes', 'Ferrari']
Matches: 2/3
